In [32]:
#fetch raw data
import os
import zipfile
import pandas as pd

# Define your paths
main_zip = r"C:\Users\DELL\Downloads\2023-citibike-tripdata.zip"
monthly_zip_dir = r"C:\Users\DELL\Downloads\2023-citibike-tripdata_temp_monthly_zips"
extract_folder = r"C:\Users\DELL\Downloads\unzipped-2023-citibike-tripdata"
merged_csv_path = r"C:\Users\DELL\Downloads\combined_citibike_2023.csv"

# Step 1: Unzip the main archive
with zipfile.ZipFile(main_zip, 'r') as zip_ref:
    zip_ref.extractall(monthly_zip_dir)

# Step 2: Extract CSVs from monthly zip files inside the extracted folder
inner_dir = os.path.join(monthly_zip_dir, "2023-citibike-tripdata")
os.makedirs(extract_folder, exist_ok=True)

for filename in os.listdir(inner_dir):
    if filename.endswith(".zip"):
        path = os.path.join(inner_dir, filename)
        with zipfile.ZipFile(path, 'r') as monthly_zip:
            monthly_zip.extractall(extract_folder)

# Step 3: Merge all CSVs into a single DataFrame
df_list = []
for f in os.listdir(extract_folder):
    if f.endswith(".csv"):
        file_path = os.path.join(extract_folder, f)
        df = pd.read_csv(file_path)
        df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv(merged_csv_path, index=False)

print(f"Combined CSV saved to: {merged_csv_path}")


C:\Users\DELL\AppData\Local\Temp\ipykernel_11576\1451669192.py:30: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11576\1451669192.py:30: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11576\1451669192.py:30: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11576\1451669192.py:30: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11576\1451669192.py:30: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\User

Combined CSV saved to: C:\Users\DELL\Downloads\combined_citibike_2023.csv


In [4]:
import pandas as pd

merged_csv_path = r"C:\Users\DELL\Downloads\combined_citibike_2023.csv"
filtered_csv_path = r"C:\Users\DELL\Downloads\filtered_top3_citibike_2023.csv"

chunk_size = 100_000  # adjust if needed
top3_counter = {}

# First pass: Count start station names across all chunks
for chunk in pd.read_csv(merged_csv_path, chunksize=chunk_size, low_memory=False):
    top_counts = chunk['start_station_name'].value_counts()
    for station, count in top_counts.items():
        top3_counter[station] = top3_counter.get(station, 0) + count

# Get top 3 station names
top3_stations = sorted(top3_counter, key=top3_counter.get, reverse=True)[:3]
print("Top 3 Locations:", top3_stations)

# Second pass: Filter only top 3 stations and save to new file
filtered_chunks = []
for chunk in pd.read_csv(merged_csv_path, chunksize=chunk_size, low_memory=False):
    filtered = chunk[chunk['start_station_name'].isin(top3_stations)]
    filtered_chunks.append(filtered)

# Concatenate filtered data and save
pd.concat(filtered_chunks).to_csv(filtered_csv_path, index=False)
print(f"Filtered CSV saved at: {filtered_csv_path}")


Top 3 Locations: ['W 21 St & 6 Ave', 'Broadway & W 58 St', 'West St & Chambers St']
Filtered CSV saved at: C:\Users\DELL\Downloads\filtered_top3_citibike_2023.csv


In [14]:
import pandas as pd

filtered_csv_path = r"C:\Users\DELL\Downloads\filtered_top3_citibike_2023.csv"
cleaned_csv_path = r"C:\Users\DELL\Downloads\cleaned_citibike_top3_2023.csv"

# Load filtered data
df = pd.read_csv(filtered_csv_path)

# Convert 'started_at' to datetime
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
df = df.dropna(subset=['started_at'])

# Feature engineering
df['date'] = df['started_at'].dt.date
df['hour'] = df['started_at'].dt.hour
df['weekday'] = df['started_at'].dt.day_name()

# Save cleaned and enriched data
df.to_csv(cleaned_csv_path, index=False)
print(f"Cleaned CSV saved at: {cleaned_csv_path}")


Cleaned CSV saved at: C:\Users\DELL\Downloads\cleaned_citibike_top3_2023.csv


In [13]:
import hopsworks
import pandas as pd

# Load the CSV
csv_path = r"C:\Users\DELL\Downloads\cleaned_citibike_top3_2023.csv"
df = pd.read_csv(csv_path, low_memory=False)

# Convert object columns to string
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].fillna("").astype(str)

#  Convert 'started_at' to datetime
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')

# Check for rows with nulls after conversion
df = df.dropna(subset=['started_at'])

# Log in
project = hopsworks.login(
    project="Citi_Bike_TripData",
    api_key_value="eVrgcmUQIaYJz4kj.QNITpj9s3ieWAofZVNhhPtsjGng1ra5ZA9BsSGNRuI6i9WLGojdUuD0i0TBKfIx1"
)

# Feature store connection
fs = project.get_feature_store()

# Create the feature group
fg = fs.create_feature_group(
    name="citibike_top3_2023",
    version=1,
    description="Top 3 Citi Bike stations from 2023, cleaned and enriched",
    primary_key=["start_station_name", "date", "hour"],
    event_time="started_at"
)

# Insert the cleaned DataFrame
fg.insert(df)

print("Data uploaded successfully to Hopsworks!")


2025-05-09 14:52:22,045 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 14:52:22,050 INFO: Initializing external client
2025-05-09 14:52:22,051 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 14:52:22,722 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225931
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1225931/fs/1213514/fg/1440646


Uploading Dataframe: 100.00% |███████████████████████| Rows 366490/366490 | Elapsed Time: 02:20 | Remaining Time: 00:00


Launching job: citibike_top3_2023_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1225931/jobs/named/citibike_top3_2023_1_offline_fg_materialization/executions
Data uploaded successfully to Hopsworks!


In [8]:
import hopsworks

# Step 1: Login to Hopsworks
project = hopsworks.login(
    project="Citi_Bike_TripData",
    api_key_value="eVrgcmUQIaYJz4kj.QNITpj9s3ieWAofZVNhhPtsjGng1ra5ZA9BsSGNRuI6i9WLGojdUuD0i0TBKfIx1"  # or use environment variable
)
fs = project.get_feature_store()

# Step 2: Get the existing feature group
fg = fs.get_feature_group(name="citibike_features_hourly", version=1)

# Step 3: Select features and create a query
query = fg.select_all()

# Step 4: Create the feature view
fv = fs.create_feature_view(
    name="citibike_hourly_feature_view",
    version=1,
    description="Feature view for Citi Bike demand prediction using hourly lag features",
    labels=["rides"],
    query=query
)

print("Feature view created successfully.")


2025-05-10 01:45:30,286 INFO: Initializing external client
2025-05-10 01:45:30,286 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 01:45:34,434 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225931
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1225931/fs/1213514/fv/citibike_hourly_feature_view/version/1
Feature view created successfully.


In [2]:
import dagshub
dagshub.init(repo_owner='SaiRishi9', repo_name='Citi_Bike_tripdata', mlflow=True)

Accessing as SaiRishi9

Initialized MLflow to track repo "SaiRishi9/Citi_Bike_tripdata"

Repository SaiRishi9/Citi_Bike_tripdata initialized!

In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

# ------------------------------
# Step 1: Load and clean dataset
# ------------------------------
df = pd.read_csv("C:/Users/DELL/Downloads/cleaned_citibike_top3_2023.csv", low_memory=False)

df["started_at"] = pd.to_datetime(df["started_at"])
df = df.sort_values("started_at")

# Use only 1 station to simplify modeling
top_station = df["start_station_name"].value_counts().idxmax()
df = df[df["start_station_name"] == top_station]

# Set rides as 1 per row, aggregate by hour
df["rides"] = 1
df = df.groupby("started_at").agg({"rides": "sum"}).reset_index()
df.set_index("started_at", inplace=True)
df = df.resample("H").sum().fillna(0)
df.reset_index(inplace=True)

# --------------------------------
# Step 2: Create lag features
# --------------------------------
for lag in range(1, 29):
    df[f"lag_{lag}"] = df["rides"].shift(lag)

df.dropna(inplace=True)

# ------------------------------
# Step 3: Split data
# ------------------------------
X = df[[f"lag_{i}" for i in range(1, 29)]]
y = df["rides"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ------------------------------
# Step 4: Start MLflow experiment
# ------------------------------
mlflow.set_experiment("citibike_modeling_experiment")
results = {}

# 1. Baseline model (Naive lag-1)
with mlflow.start_run(run_name="Baseline_Lag1"):
    y_pred = X_test["lag_1"]
    mae = mean_absolute_error(y_test, y_pred)
    mlflow.log_param("model", "Baseline")
    mlflow.log_metric("MAE", mae)
    results["Baseline Lag-1"] = mae

# 2. LightGBM with all 28 lags
with mlflow.start_run(run_name="LightGBM_All28Lags"):
    model = lgb.LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mlflow.log_param("model", "LightGBM-28-lags")
    mlflow.log_metric("MAE", mae)
    mlflow.sklearn.log_model(model, "model")
    results["LightGBM - 28 Lags"] = mae

# 3. LightGBM with Top 10 selected lags
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

selector = SelectKBest(score_func=f_regression, k=10)
X_selected = selector.fit_transform(X_scaled, y)
selected_indices = selector.get_support(indices=True)
selected_columns = [f"lag_{i+1}" for i in selected_indices]

X_train_top = X_train[selected_columns]
X_test_top = X_test[selected_columns]
y_train_top = y_train
y_test_top = y_test

with mlflow.start_run(run_name="LightGBM_Top10Lags"):
    model = lgb.LGBMRegressor(random_state=42)
    model.fit(X_train_top, y_train_top)
    y_pred = model.predict(X_test_top)
    mae = mean_absolute_error(y_test_top, y_pred)
    mlflow.log_param("model", "LightGBM-Top10")
    mlflow.log_metric("MAE", mae)
    mlflow.sklearn.log_model(model, "model")
    results["LightGBM - Top 10 Lags"] = mae

# ------------------------------
# Step 5: Print results
# ------------------------------
print("\n Modeling complete. MAE summary:")
for model_name, mae_val in results.items():
    print(f"{model_name}: MAE = {mae_val:.3f}")


🏃 View run Baseline_Lag1 at: https://dagshub.com/SaiRishi9/Citi_Bike_tripdata.mlflow/#/experiments/0/runs/e2550eb01dcf46feb1549baad0376e4f
🧪 View experiment at: https://dagshub.com/SaiRishi9/Citi_Bike_tripdata.mlflow/#/experiments/0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000669 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2240
[LightGBM] [Info] Number of data points in the train set: 6985, number of used features: 28
[LightGBM] [Info] Start training from score 16.132283


2025/05/09 21:40:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_All28Lags at: https://dagshub.com/SaiRishi9/Citi_Bike_tripdata.mlflow/#/experiments/0/runs/0dfd858853df4d9d9efa0f2a96c24edc
🧪 View experiment at: https://dagshub.com/SaiRishi9/Citi_Bike_tripdata.mlflow/#/experiments/0
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000810 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 800
[LightGBM] [Info] Number of data points in the train set: 6985, number of used features: 10
[LightGBM] [Info] Start training from score 16.132283


2025/05/09 21:40:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_Top10Lags at: https://dagshub.com/SaiRishi9/Citi_Bike_tripdata.mlflow/#/experiments/0/runs/ab1da64f1e304ee1b546796a64c23ba3
🧪 View experiment at: https://dagshub.com/SaiRishi9/Citi_Bike_tripdata.mlflow/#/experiments/0

 Modeling complete. MAE summary:
Baseline Lag-1: MAE = 5.743
LightGBM - 28 Lags: MAE = 4.023
LightGBM - Top 10 Lags: MAE = 4.183


In [6]:
import mlflow.pyfunc

model_uri = "models:/citibike_demand_model/1"
model = mlflow.pyfunc.load_model(model_uri)

# Sample inference
sample_input = X_test.iloc[[0]]  # replace with your own sample
prediction = model.predict(sample_input)
print("Predicted rides:", prediction)


Predicted rides: [1.62019053]


In [10]:
# src/feature_pipeline.py
import pandas as pd
import hopsworks

# Step 1: Load raw Citi Bike trip data
csv_path = r"C:\Users\DELL\Downloads\cleaned_citibike_top3_2023.csv"
df = pd.read_csv(csv_path, low_memory=False)

# Step 2: Convert started_at to datetime and aggregate to hourly ride counts
df['started_at'] = pd.to_datetime(df['started_at'])
df['rides'] = 1  # each row is a ride
df = df.set_index('started_at').resample('H').sum().reset_index()

# Step 3: Create lag features
for lag in range(1, 29):
    df[f'lag_{lag}'] = df['rides'].shift(lag)

# Step 4: Drop missing rows due to lagging
df.dropna(inplace=True)

# Step 5: Extract additional columns for keys
df['date'] = df['started_at'].dt.floor('D')  # datetime-compatible type
df['hour'] = df['started_at'].dt.hour
df['start_station_name'] = "Top_Station"  # replace with actual if available

# Step 6: Keep only relevant columns
features = [f'lag_{i}' for i in range(1, 29)] + ['date', 'hour', 'start_station_name', 'rides']
df = df[features]

# Step 7: Connect to Hopsworks
project = hopsworks.login(
    project="Citi_Bike_TripData",
    api_key_value="eVrgcmUQIaYJz4kj.QNITpj9s3ieWAofZVNhhPtsjGng1ra5ZA9BsSGNRuI6i9WLGojdUuD0i0TBKfIx1"
)
fs = project.get_feature_store()

# Step 8: Create or get the feature group
fg = fs.get_or_create_feature_group(
    name="citibike_features_hourly",
    version=1,
    primary_key=["start_station_name", "date", "hour"],
    event_time="date",
    description="Hourly lag features for Citi Bike demand prediction"
)

# Step 9: Insert data in chunks to avoid disconnect errors
batch_size = 5000
for i in range(0, len(df), batch_size):
    chunk = df.iloc[i:i+batch_size]
    fg.insert(chunk, wait=True)
    print(f"Inserted rows {i} to {i+len(chunk)-1}")

print("Feature group successfully inserted into Hopsworks.")


2025-05-09 21:16:30,172 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 21:16:30,181 INFO: Initializing external client
2025-05-09 21:16:30,181 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 21:16:30,835 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1225931


Uploading Dataframe: 100.00% |███████████████████████████| Rows 5000/5000 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: citibike_features_hourly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1225931/jobs/named/citibike_features_hourly_1_offline_fg_materialization/executions
2025-05-09 21:16:47,662 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-09 21:16:50,759 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-09 21:18:23,435 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-05-09 21:18:23,513 INFO: Waiting for log aggregation to finish.
2025-05-09 21:18:31,831 INFO: Execution finished successfully.
Inserted rows 0 to 4999


Uploading Dataframe: 100.00% |███████████████████████████| Rows 3817/3817 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: citibike_features_hourly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1225931/jobs/named/citibike_features_hourly_1_offline_fg_materialization/executions
2025-05-09 21:18:47,160 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-09 21:18:50,264 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-09 21:20:16,623 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-05-09 21:20:22,792 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-05-09 21:20:23,021 INFO: Waiting for log aggregation to finish.
2025-05-09 21:20:23,023 INFO: Execution finished successfully.
Inserted rows 5000 to 8816
Feature group successfully inserted into Hopsworks.
